In [1]:
import os
import cv2
import math

# --- File paths ---
input_video_path = "../../videos/Outdoor/top/split/IMG_0109_3.MOV"
mot_path         = "../../BoT-SORT_outputs/Outdoor/filtered/IMG_0109_3.txt"
att_path         = "../../BoT-SORT_outputs/Outdoor/transformed/with_jersey_number/with_team/IMG_0109_3.txt"
pred_video_path  = "../../videos/Outdoor/top/minimap/IMG_0109_3_pred.mp4"
gt_video_path    = "../../videos/Outdoor/top/minimap/IMG_0109_3_GT.mp4"
output_dir       = "../../videos/Outdoor/top/minimap"
os.makedirs(output_dir, exist_ok=True)

# --- get_color function ---
def get_color(attribute):
    """
    属性に応じた色を返します。
    属性が 'O' で始まるならピンク (BGR: (180, 105, 255))、
    'D' で始まるなら紫 (BGR: (128, 0, 128))、
    それ以外の場合は黒を返します。
    """
    if attribute.startswith('O'):
        return (180, 105, 255)
    elif attribute.startswith('D'):
        return (128, 0, 128)
    else:
        return (0, 0, 0)

# --- File read functions ---
def load_mot_file(path):
    mot_dict = {}
    with open(path, "r") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            parts = [p.strip() for p in line.split(",")]
            if len(parts) < 6:
                continue
            try:
                frame = int(float(parts[0]))
                track_id = int(float(parts[1]))
                x = float(parts[2])
                y = float(parts[3])
                w = float(parts[4])
                h = float(parts[5])
            except Exception as e:
                print(f"parse error: {line}")
                continue
            if frame not in mot_dict:
                mot_dict[frame] = {}
            mot_dict[frame][track_id] = (x, y, w, h)
    return mot_dict

def load_att_file(path):
    att_dict = {}
    with open(path, "r") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            parts = [p.strip() for p in line.split(",")]
            if len(parts) < 5:
                continue
            try:
                frame = int(float(parts[0]))
                track_id = int(float(parts[1]))
                x = float(parts[2])
                y = float(parts[3])
                attribute = parts[-1]
            except Exception as e:
                print(f"Attribute file parsing error: {line}")
                continue
            if frame not in att_dict:
                att_dict[frame] = {}
            att_dict[frame][track_id] = (x, y, attribute)
    return att_dict

# --- Load MOT and Attribute data ---
mot_data = load_mot_file(mot_path)
att_data = load_att_file(att_path)

# --- Open videos ---
cap = cv2.VideoCapture(input_video_path)
pred_cap = cv2.VideoCapture(pred_video_path)
gt_cap = cv2.VideoCapture(gt_video_path)

if not cap.isOpened():
    print(f"Could not open video: {input_video_path}")
    exit()

main_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
main_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

fourcc = cv2.VideoWriter_fourcc(*"mp4v")
video_filename = os.path.basename(input_video_path)
output_video_path = os.path.join(output_dir, video_filename)
out = cv2.VideoWriter(output_video_path, fourcc, fps, (main_width, main_height))

# Set M to 1/5 of the width of the main video and compute L accordingly
M = main_width / 5.0
L = (4/3.0) * M

# Resize the mini-map video to width M while keeping aspect ratio
ret_pred, first_pred = pred_cap.read()
if not ret_pred:
    print(f"Failure to acquire frames for Prediction video: {pred_video_path}")
    exit()
orig_pred_h, orig_pred_w = first_pred.shape[:2]
r = orig_pred_h / orig_pred_w
H = M * r
# Reset frame positions
pred_cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
gt_cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

pred_x = int(L)
gt_x   = int(L + M + (M/3.0))
label_space = 50
overlay_y = main_height - int(H + label_space)

# --- Font settings for labels ---
font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 2
font_thickness = 5
label_color = (0, 0, 0)  # Black

frame_index = 0
while True:
    ret_main, main_frame = cap.read()
    if not ret_main:
        break

    # MOT frame index and corresponding attribute frame index
    current_mot_frame = frame_index
    current_att_frame = frame_index + 1
    if current_mot_frame in mot_data:
        for track_id, bbox in mot_data[current_mot_frame].items():
            x, y, w, h = bbox
            a = w / 2
            b = a / 5
            cx = int(x + w/2)
            cy = int(y + h + b/2)
            # Get attribute if available, else empty string
            attribute = ""
            if current_att_frame in att_data and track_id in att_data[current_att_frame]:
                attribute = att_data[current_att_frame][track_id][2]
            color = get_color(attribute)
            if color == (0, 0, 0):
                continue
            cv2.ellipse(main_frame, (cx, cy), (int(a), int(b)), 0, 330, 570, color, 3)
            text_pos = (int(x+25), int(y - 10))
            cv2.putText(main_frame, attribute, text_pos,
                        cv2.FONT_HERSHEY_SIMPLEX, 1.3, color, 3, cv2.LINE_AA)
    
    # --- mini-map overlay ---
    ret_pred, frame_pred = pred_cap.read()
    ret_gt, frame_gt = gt_cap.read()
    if ret_pred and ret_gt:
        frame_pred_resized = cv2.resize(frame_pred, (int(M), int(H)), interpolation=cv2.INTER_LINEAR)
        frame_gt_resized   = cv2.resize(frame_gt,   (int(M), int(H)), interpolation=cv2.INTER_LINEAR)
        alpha = 0.8
        
        roi_pred = main_frame[overlay_y:overlay_y+int(H), pred_x:pred_x+int(M)]
        blended_pred = cv2.addWeighted(roi_pred, 1 - alpha, frame_pred_resized, alpha, 0)
        main_frame[overlay_y:overlay_y+int(H), pred_x:pred_x+int(M)] = blended_pred
        
        roi_gt = main_frame[overlay_y:overlay_y+int(H), gt_x:gt_x+int(M)]
        blended_gt = cv2.addWeighted(roi_gt, 1 - alpha, frame_gt_resized, alpha, 0)
        main_frame[overlay_y:overlay_y+int(H), gt_x:gt_x+int(M)] = blended_gt
        
        # --- Label Drawing ---
        pred_label_x = pred_x + int(M/2)
        gt_label_x = gt_x + int(M/2)
        label_y = overlay_y + int(H) + 42 
        cv2.putText(main_frame, "Prediction", (pred_label_x - 150, label_y), font, font_scale, label_color, font_thickness)
        cv2.putText(main_frame, "Ground Truth", (gt_label_x - 190, label_y), font, font_scale, label_color, font_thickness)
            
    out.write(main_frame)
    frame_index += 1

cap.release()
pred_cap.release()
gt_cap.release()
out.release()
print(f"Processing is complete. Output: {output_video_path}")

Processing is complete. Output: ../../videos/Outdoor/top/minimap/IMG_0109_3.MOV
